In [27]:
import os
import json
import pandas as pd

results_dir = "../data/results/longmemeval_short_RAG_azure_gpt-5-mini_azure_gpt-5-mini"
json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]

results = []
for filename in json_files:
    filepath = os.path.join(results_dir, filename)
    with open(filepath, "r") as f:
        data = json.load(f)
        results.append(data)

df = pd.DataFrame(results)
df

,question_id,question,predicted_answer,answer,answer_is_correct
0,2ce6a0f2,How many different art-related events did I at...,Three.,4,False
1,gpt4_93159ced_abs,How long have I been working before I started ...,I can't determine that from the provided evide...,The information provided is not enough. From t...,True
2,ce6d2d27,What day of the week do I take a cocktail-maki...,Friday,Friday,True
3,b5ef892d,How many days did I spend on camping trips in ...,5 days.,8 days.,False
4,gpt4_f49edff3,Which three events happened in the order from ...,First: helped your friend prepare the nursery ...,"First, I helped my friend prepare the nursery,...",True
5,gpt4_85da3956,How many weeks ago did I attend the 'Summer Ni...,Three weeks ago.,3 weeks ago,True
6,2788b940,How many fitness classes do I attend in a typi...,Five classes per week.,5,True
7,5a7937c8,How many days did I spend participating in fai...,You spent time on faith-related activities on ...,3 days.,False
8,0e4e4c46,What is my current highest score in Ticket to ...,Your current highest score is 132 points.,132 points,True
9,618f13b2,How many times have I worn my new black Conver...,You’ve worn them six times.,six,True


In [28]:
print(df['answer_is_correct'].mean())

0.7894736842105263


In [29]:
for _, row in df.iterrows():
    answer = row['answer']
    predicted_answer = row['predicted_answer']
    correct = row['answer_is_correct']
    print(f"Question: {row['question']}")
    print(f"Answer: {answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Correct: {correct}")
    print("\n")


Question: How many different art-related events did I attend in the past month?
Answer: 4
Predicted Answer: Three.
Correct: False


Question: How long have I been working before I started my current job at Google?
Answer: The information provided is not enough. From the information provided, You haven't started working at Google yet.
Predicted Answer: I can't determine that from the provided evidence. The transcript only says you've been at NovaTech for about 4 years and 3 months; there is no mention of a job at Google or any dates for when you might have started at Google.
Correct: True


Question: What day of the week do I take a cocktail-making class?
Answer: Friday
Predicted Answer: Friday
Correct: True


Question: How many days did I spend on camping trips in the United States this year?
Answer: 8 days.
Predicted Answer: 5 days.
Correct: False


Question: Which three events happened in the order from first to last: the day I helped my friend prepare the nursery, the day I helped m

In [30]:
with open("../data/longmemeval/longmemeval_s_cleaned.json", "r", encoding="utf-8") as f:
    longmemeval_df = pd.DataFrame(json.load(f)).reset_index(drop=True)

In [31]:
df.merge(longmemeval_df[["question_id", "question_type"]], on="question_id", how="inner").groupby(
    "question_type"
).agg(
    answer_is_correct_mean=("answer_is_correct", "mean"), count=("answer_is_correct", "count")
).sort_values(
    by="answer_is_correct_mean", ascending=False
).reset_index()

,question_type,answer_is_correct_mean,count
0,single-session-assistant,1.000000,5
1,single-session-preference,1.000000,1
2,single-session-user,1.000000,3
3,temporal-reasoning,0.857143,7
4,knowledge-update,0.800000,10
5,multi-session,0.583333,12


In [32]:
longmemeval_df['question_type'].value_counts()

question_type
multi-session                133
temporal-reasoning           133
knowledge-update              78
single-session-user           70
single-session-assistant      56
single-session-preference     30
Name: count, dtype: int64